In [1]:
import uproot
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import umap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from tqdm import tqdm
import torch.cuda.amp as amp

In [2]:
# Load datasets
# #paths to samples
zero_t_samples = [
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/WW.Flz.part1.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/WW.Flz.part2.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/Wjets-incl.INb.part1.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/Wjets-incl.INb.part2.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/Wjets-incl.INb.part3.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/Wjets-incl.INb.part4.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/Wjets-incl.INb.part5.root',
]

one_t_samples = [
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/t-channel_tbar_4f.leB.part1.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/t-channel_tbar_4f.leB.part2.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/t-channel_top_4f.qlh.part1.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/t-channel_top_4f.qlh.part2.root',
]

two_t_samples = [
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/ttbar_doublelep.KQP.part1.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/ttbar_doublelep.KQP.part2.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/ttbar_doublelep.KQP.part3.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/ttbar_doublelep.KQP.part4.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/ttbar_semilep.vRt.part1.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/ttbar_semilep.vRt.part2.root',
]

three_t_samples = [
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/TTTJ_UL16Summer20_RoO.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/TTTW_UL16Summer20_CgT.root',
]

four_t_samples = [
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/TTTT_UL16Summer20_eIE.part1.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/TTTT_UL16Summer20_eIE.part2.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/TTTT_UL16Summer20_eIE.part3.root',
    '/scratch/pvolkov/DM/samples/MultiLepton/grid/TTTT_UL16Summer20_eIE.part4.root',
]

# create dataframe
total_data = []
for i, samples in tqdm(enumerate([zero_t_samples,one_t_samples,two_t_samples,three_t_samples,four_t_samples])):
    list_of_samples = []
    for path in tqdm(samples):
        print('processing',i, path)
        file = uproot.open(path)
        a = file['LHE'].arrays(file['LHE'].keys(), library="np")
        df = pd.DataFrame(a)
        df = df.loc[:, ~df.columns.str.startswith('SP_')]
        list_of_samples.append(df)
        print('successful')
    df_sample = pd.concat(list_of_samples).reset_index()
    df_sample['class'] = i
    total_data.append(df_sample)
data = pd.concat(total_data).reset_index()
data = data.drop(columns = ['level_0', 'index'])
print('shape:', data.shape)

0it [00:00, ?it/s]

processing 0 /scratch/pvolkov/DM/samples/MultiLepton/grid/WW.Flz.part1.root


successful
processing 0 /scratch/pvolkov/DM/samples/MultiLepton/grid/WW.Flz.part2.root


successful
processing 0 /scratch/pvolkov/DM/samples/MultiLepton/grid/Wjets-incl.INb.part1.root


successful
processing 0 /scratch/pvolkov/DM/samples/MultiLepton/grid/Wjets-incl.INb.part2.root


...

successful
processing 4 /scratch/pvolkov/DM/samples/MultiLepton/grid/TTTT_UL16Summer20_eIE.part4.root


100%|██████████| 4/4 [00:35<00:00,  8.86s/it]

successful



5it [04:53, 58.63s/it]


shape: (8198428, 126)


In [5]:
particle_types = {
    "BJet": 4,
    "Jet": 12,
    "Lep": 4
}

particle_features = ["Pt", "Eta", "Phi", "Px", "Py", "Pz"]

particle_cols = []
for p_type, count in particle_types.items():
    for i in range(1, count + 1):
        for feat in particle_features:
            col_name = f"{feat}_{p_type}{i}"
            particle_cols.append(col_name)

global_cols = ["N_J", "N_BJ", "N_Nu", "N_Lep", "MeT"]

label_col = "class"

train_df, val_df = train_test_split(data, test_size=0.2, stratify=data['class'], shuffle=True, random_state=42)

In [6]:

class AEDataset(Dataset):
    def __init__(self, df, particle_cols, global_cols, num_particles=20, particle_feat_dim=6):
        self.df = df.reset_index(drop=True)
        self.particle_cols = particle_cols
        self.global_cols = global_cols
        self.num_particles = num_particles
        self.particle_feat_dim = particle_feat_dim

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        
        particles = torch.tensor(row[self.particle_cols].values.reshape(self.num_particles, self.particle_feat_dim), dtype=torch.float32)

        
        global_feats = torch.tensor(row[self.global_cols].values, dtype=torch.float32)

        return particles, global_feats


In [9]:
class AEEncoder(nn.Module):
    def __init__(self, num_particles=20, particle_feat_dim=6, global_feat_dim=5, emb_dim=64, num_heads=4):
        super().__init__()
        self.particle_proj = nn.Linear(particle_feat_dim, emb_dim)
        self.attn = nn.MultiheadAttention(embed_dim=emb_dim, num_heads=num_heads, batch_first=True)
        self.particle_pool = nn.AdaptiveAvgPool1d(1)
        self.global_net = nn.Sequential(
            nn.Linear(global_feat_dim, emb_dim),
            nn.LayerNorm(emb_dim),
            nn.GELU(),
            nn.Linear(emb_dim, emb_dim)
        )
        self.combined_proj = nn.Linear(emb_dim * 2, emb_dim)

    def forward(self, particles, global_feats):
        x = self.particle_proj(particles)
        attn_out, _ = self.attn(x, x, x)
        pooled_particles = self.particle_pool(attn_out.transpose(1, 2)).squeeze(-1)
        global_emb = self.global_net(global_feats)
        combined = torch.cat([pooled_particles, global_emb], dim=1)
        emb = self.combined_proj(combined)
        return emb

class AEDecoder(nn.Module):
    def __init__(self, num_particles=20, particle_feat_dim=6, global_feat_dim=5, emb_dim=64):
        super().__init__()
        output_dim = num_particles * particle_feat_dim + global_feat_dim
        self.decoder = nn.Sequential(
            nn.Linear(emb_dim, 256),
            nn.GELU(),
            nn.Linear(256, output_dim)
        )
        self.num_particles = num_particles
        self.particle_feat_dim = particle_feat_dim
        self.global_feat_dim = global_feat_dim

    def forward(self, emb):
        out = self.decoder(emb)
        particles = out[:, :self.num_particles * self.particle_feat_dim]
        global_feats = out[:, self.num_particles * self.particle_feat_dim:]
        particles = particles.view(-1, self.num_particles, self.particle_feat_dim)
        return particles, global_feats

class Autoencoder(nn.Module):
    def __init__(self, num_particles=20, particle_feat_dim=6, global_feat_dim=5, emb_dim=64, num_heads=4):
        super().__init__()
        self.encoder = AEEncoder(num_particles, particle_feat_dim, global_feat_dim, emb_dim, num_heads)
        self.decoder = AEDecoder(num_particles, particle_feat_dim, global_feat_dim, emb_dim)

    def forward(self, particles, global_feats):
        emb = self.encoder(particles, global_feats)
        particles_recon, global_recon = self.decoder(emb)
        return particles_recon, global_recon, emb

In [7]:
train_dataset = AEDataset(train_df, particle_cols, global_cols, num_particles=20, particle_feat_dim=6)
val_dataset = AEDataset(val_df, particle_cols, global_cols, num_particles=20, particle_feat_dim=6)

In [8]:
def loss_fn(particles_recon, particles_true, global_recon, global_true):
    loss_particles = F.mse_loss(particles_recon, particles_true)
    loss_global = F.mse_loss(global_recon, global_true)
    return loss_particles + loss_global

In [11]:
def train_autoencoder(model, train_dataset, val_dataset, config):
    device = config["device"]
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

    train_loader = DataLoader(
        train_dataset,
        batch_size=config["batch_size"],
        shuffle=True,
        num_workers=6,
        pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config["batch_size"],
        shuffle=False,
        num_workers=6,
        pin_memory=True
    )

    best_val_loss = float("inf")
    train_losses, val_losses = [], []

    for epoch in range(config["epochs"]):
        model.train()
        total_loss = 0
        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{config['epochs']}")

        for batch in loop:
            particles, global_feats = batch  # вот здесь батч
            particles = particles.to(device, non_blocking=True)
            global_feats = global_feats.to(device, non_blocking=True)

            optimizer.zero_grad()
            particles_recon, global_recon, emb = model(particles, global_feats)
            loss = loss_fn(particles_recon, particles, global_recon, global_feats)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                particles, global_feats = batch
                particles = particles.to(device, non_blocking=True)
                global_feats = global_feats.to(device, non_blocking=True)

                particles_recon, global_recon, emb = model(particles, global_feats)
                loss = loss_fn(particles_recon, particles, global_recon, global_feats)

                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        print(f"[{epoch+1}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), config["model_save_path"])
            print(">> Best model saved.")

    plt.figure(figsize=(8, 5))
    plt.plot(train_losses, label="Train")
    plt.plot(val_losses, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Autoencoder Training Loss")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(config["loss_plot_path"])
    plt.close()

In [ ]:
config = {
    "device": "cuda" if torch.cuda.is_available() else "cpu",
    "lr": 1e-4,
    "batch_size": 256,
    "epochs": 20,
    "model_save_path": "best_ae_model.pt",
    "loss_plot_path": "ae_loss_curve.png"
}
model = Autoencoder(
    num_particles=20,
    particle_feat_dim=6,
    global_feat_dim=5,
    emb_dim=64,
    num_heads=4
)

model.to('cuda')

Autoencoder(
  (encoder): AEEncoder(
    (particle_proj): Linear(in_features=6, out_features=64, bias=True)
    (attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
    )
    (particle_pool): AdaptiveAvgPool1d(output_size=1)
    (global_net): Sequential(
      (0): Linear(in_features=5, out_features=64, bias=True)
      (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (2): GELU(approximate='none')
      (3): Linear(in_features=64, out_features=64, bias=True)
    )
    (combined_proj): Linear(in_features=128, out_features=64, bias=True)
  )
  (decoder): AEDecoder(
    (decoder): Sequential(
      (0): Linear(in_features=64, out_features=256, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=256, out_features=125, bias=True)
    )
  )
)

In [12]:
train_autoencoder(model, train_dataset, val_dataset, config)

Epoch 1/20:   1%|          | 557/51241 [00:28<43:26, 19.44it/s, loss=4.45e+3]  


KeyboardInterrupt: 